# 02 — RNN pooling fix (SmartWatch Gestures)

# Doel
De baseline gebruikt een naïeve sequentierepresentatie door alleen de **laatste timestep** van de (gepaddede) sequentie te gebruiken voor classificatie.  
Omdat sequenties **variabele lengte** hebben en op batch-niveau worden **gepaddede**, kan deze keuze informatieverlies veroorzaken of padding-effecten introduceren.

# Hypothese
Als we de sequentierepresentatie vervangen door **pooling over tijd** (bijv. gemiddelde over alle timesteps), dan wordt het model robuuster voor variabele sequentielengtes en verwachten we een **hogere en/of stabielere validatie-accuracy**.

# Experiment
We houden dataset, preprocessor, TrainerSettings en optimizer gelijk, en vergelijken:
- Baseline representatie: laatste timestep
- Pooling representatie: mean pooling over tijd

Resultaten worden gelogd in MLflow zodat runs direct vergelijkbaar zijn.


In [17]:
import sys
from pathlib import Path

# Vind repo-root door omhoog te lopen totdat we pyproject.toml of .git vinden
p = Path().resolve()
while not (p / "pyproject.toml").exists() and not (p / ".git").exists():
    if p.parent == p:
        raise RuntimeError("Repo root not found (no pyproject.toml/.git).")
    p = p.parent

PROJECT_ROOT = p
sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)


Project root added to sys.path: /home/mischatrader1/MADS/MADS-MachineLearning-course


In [18]:
from pathlib import Path

import torch

from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

In [19]:
import torch

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Override naar CPU voor kleine RNN-modellen indien gewenst
device = torch.device("cpu")

device


Using CPU


device(type='cpu')

In [20]:
# Data + preprocessor (identiek aan notebook 01)
preprocessor = PaddedPreprocessor()

gestures_factory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gestures_factory.create_datastreamer(batchsize=32, preprocessor=preprocessor)

train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

len(train), len(valid)

2026-01-08 08:57:02.624 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/mischatrader1/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 2192.95it/s]


(81, 20)

In [21]:
accuracy = Accuracy()

settings = TrainerSettings(
    epochs=30,  # bewust korter dan "100": dit notebook is een gerichte vergelijking
    metrics=[accuracy],
    logdir=Path("gestures_ex3"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs={
        "save": False,
        "verbose": True,
        "patience": 5,
        "delta": 0.0,
    },
)
settings

epochs: 30
metrics: [Accuracy]
logdir: gestures_ex3
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [22]:
from src_rnn.models import RNNConfig
from src_rnn.run_experiment import run_experiment

In [23]:
# Run 1: Baseline representatie (laatste timestep)
config = RNNConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)

_ = run_experiment(
    model_name="gru_last",                 # baseline: last-step representatie
    config=config,
    settings=settings,
    trainstreamer=train.stream(),
    validstreamer=valid.stream(),
    device=device,
    experiment_name="gestures-ex3",
    run_name="02_baseline_gru_last_h64_l1",
)


2026-01-08 08:57:04.550 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures_ex3/20260108-085704
2026-01-08 08:57:04.552 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 48.27it/s]
2026-01-08 08:57:06.371 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.9331 test 2.7137 metric ['0.1391']
100%|██████████| 81/81 [00:01<00:00, 47.92it/s]
2026-01-08 08:57:08.207 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.3489 test 2.1778 metric ['0.2453']
100%|██████████| 81/81 [00:01<00:00, 51.46it/s]
2026-01-08 08:57:09.920 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.0821 test 1.9334 metric ['0.2953']
100%|██████████| 81/81 [00:01<00:00, 62.48it/s]
2026-01-08 08:57:11.326 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.8853 test 1.7203 metric ['0.4281']
100%|██████████| 81/81 [00:01<00:00, 59.26it/s]
2026-01-08 08:57:1

In [24]:
# Run 2: Pooling fix (mean pooling over tijd)
config = RNNConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)

_ = run_experiment(
    model_name="gru_mean",                 # fix: mean-pooling representatie
    config=config,
    settings=settings,
    trainstreamer=train.stream(),
    validstreamer=valid.stream(),
    device=device,
    experiment_name="gestures-ex3",
    run_name="02_fix_gru_meanpool_h64_l1",
)

2026-01-08 08:57:49.183 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures_ex3/20260108-085749
2026-01-08 08:57:49.185 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 63.30it/s]
2026-01-08 08:57:50.576 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.8139 test 2.5186 metric ['0.2422']
100%|██████████| 81/81 [00:01<00:00, 59.81it/s]
2026-01-08 08:57:52.039 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.2238 test 1.9370 metric ['0.4656']
100%|██████████| 81/81 [00:01<00:00, 63.60it/s]
2026-01-08 08:57:53.487 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.6636 test 1.4192 metric ['0.5938']
100%|██████████| 81/81 [00:01<00:00, 52.50it/s]
2026-01-08 08:57:55.208 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.2796 test 1.1235 metric ['0.7000']
100%|██████████| 81/81 [00:01<00:00, 49.72it/s]
2026-01-08 08:57:5

# Resultaten & Reflectie — RNN Pooling Fix

In dit notebook heb ik onderzocht hoe de representatie van een RNN-output invloed heeft op de prestaties bij gesture-classificatie. Ik heb twee varianten vergeleken:

1. GRU – last timestep (gru_last)
2. GRU – mean pooling over tijd (gru_mean)

Beide modellen zijn getraind met identieke data, hyperparameters en trainingsinstellingen. De resultaten zijn gelogd en vergeleken met MLflow.

# Resultaten (MLflow)

- GRU last-step:
-   Validatie-accuracy ≈ 0.96–0.97
-   Hogere validatie-loss

- GRU mean pooling:
-   Validatie-accuracy ≈ 0.98
-   Lagere en stabielere validatie-loss

#Interpretatie
Het gebruik van alleen de laatste timestep blijkt gevoelig voor padding en variabele sequentielengtes. Relevante informatie kan zich eerder in de sequentie bevinden en wordt in dat geval deels genegeerd. Mean pooling over de tijdsdimensie combineert informatie uit alle timesteps en levert daardoor een robuustere representatie op van de volledige sequentie.

# Conclusie
Mean pooling over de RNN-output leidt tot:
- Betere generalisatie
- Hogere validatie-accuracy
- Meer stabiel trainingsgedrag

Deze pooling-fix vormt een duidelijke verbetering ten opzichte van de naïeve last-step representatie en dient als sterke baseline voor verdere experimenten.